In [1]:
import pandas as pd
import os
import csv
import re
from groq import Groq
import time

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
def safe_filename(text, max_length=40):
    filename = re.sub(r'[^a-zA-Z0-9_-]', '_', text)
    filename = filename[:max_length].rstrip('_')
    return filename

def parse_markdown_table(markdown_text):
    lines = markdown_text.strip().splitlines()
    table_lines = [line for line in lines if "|" in line]
    if len(table_lines) > 1 and "---" in table_lines[1]:
        table_lines.pop(1)
    table = [ [cell.strip() for cell in row.split("|")[1:-1]] for row in table_lines]
    return table

In [4]:
df = pd.read_csv("../in_scope_benchmark_questions.csv")
df

,Question
0,What are the approved drugs for triple-negativ...
1,List all ALK inhibitors used for treating non-...
2,What is the mechanism of action of nivolumab?
3,Drugs with approval status for hepatitis C vir...
4,Biomarkers associated with the PI3KAKT pathwa...
...,...
95,What are the approved drugs for EGFR exon 20 i...
96,List all TRK inhibitors used for treating NTRK...
97,What is the mechanism of action of darolutamide?
98,Which genes are targeted by monoclonal antibod...


In [5]:
system_prompt = (
    "Return the results in a table. For each entry, include a citation of the source within the table. "
    "Each answer row should contain only a single atomic fact per cellno multiple values, no commas, "
    "no slashes (/), and no other separators within any cell. "
    "Return only unique rows; do not include duplicate entries in the table."
)

In [6]:
models = ["llama-3.3-70b-versatile"]
trial_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
root_folder = "result"

In [7]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [8]:
client = Groq(api_key=groq_api_key)

In [9]:
os.makedirs(root_folder, exist_ok=True)

global_metrics = []

for model in models:

    try:
        model_folder = os.path.join(root_folder, safe_filename(model))
        os.makedirs(model_folder, exist_ok=True)

        for trial_number in trial_numbers:
            try:
                trial_folder = os.path.join(model_folder, f"trial_{trial_number}")
                os.makedirs(trial_folder, exist_ok=True)

                for i in range(df.shape[0]):

                    try:
                        question = df.loc[i, 'Question']
                        print(f"Processing: {question} | Model: {model} | Trial: {trial_number}")

                        start_time = time.time()
                        completion = client.chat.completions.create(
                            messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": question}
                            ],
                            model=model,
                            temperature=0,
                            max_completion_tokens=8192,
                            stream=False,
                            stop=None
                        )
                        elapsed = time.time() - start_time

                        response_text = completion.choices[0].message.content
                        table = parse_markdown_table(response_text)
                        num_rows = len(table) - 1 if len(table) > 1 else 0

                        base_filename = safe_filename(question.lower().replace(" ", "_"))
                        excel_filename = os.path.join(trial_folder, f"{base_filename}.xlsx")

                        df_table = pd.DataFrame(table[1:], columns=table[0]) if num_rows > 0 else pd.DataFrame()
                        df_metrics = pd.DataFrame(
                            {
                                "Metric": ["Rows Retrieved", "Time Taken (s)"],
                                "Value": [num_rows, round(elapsed, 4)],
                                "trial": trial_number,
                                "model": model,
                                "question": question
                            }
                        )

                        with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                            if not df_table.empty:
                                df_table.to_excel(writer, sheet_name="Table", index=False)
                            df_metrics.to_excel(writer, sheet_name="Metrics", index=False)

                        # For the global summary, add one row per question/trial/model
                        global_metrics.append({
                            "question": question,
                            "model": model,
                            "trial": trial_number,
                            "rows_retrieved": num_rows,
                            "time_taken_s": round(elapsed, 4)
                        })

                        print(f"Saved to {excel_filename}")

                    except:

                        print("----- Some error occured-----")
                        print(f"Processing: {question} | Model: {model} | Trial: {trial_number}")

                        continue

                
            except:

                print("----- Some error occured-----")
                print(f"Processing: {question} | Model: {model} | Trial: {trial_number}")

                continue


    except:

        print("----- Some error occured-----")
        print(f"Processing: {question} | Model: {model} | Trial: {trial_number}")

        continue



Processing: What are the approved drugs for triple-negative breast cancer? | Model: llama-3.3-70b-versatile | Trial: 0
Saved to result/llama-3_3-70b-versatile/trial_0/what_are_the_approved_drugs_for_triple-n.xlsx
Processing: List all ALK inhibitors used for treating non-small cell lung cancer. | Model: llama-3.3-70b-versatile | Trial: 0
Saved to result/llama-3_3-70b-versatile/trial_0/list_all_alk_inhibitors_used_for_treatin.xlsx
Processing: What is the mechanism of action of nivolumab? | Model: llama-3.3-70b-versatile | Trial: 0
Saved to result/llama-3_3-70b-versatile/trial_0/what_is_the_mechanism_of_action_of_nivol.xlsx
Processing: Drugs with approval status for hepatitis C virus infection. | Model: llama-3.3-70b-versatile | Trial: 0
Saved to result/llama-3_3-70b-versatile/trial_0/drugs_with_approval_status_for_hepatitis.xlsx
Processing: Biomarkers associated with the PI3KAKT pathway in breast cancer. | Model: llama-3.3-70b-versatile | Trial: 0
Saved to result/llama-3_3-70b-versatile

In [10]:

# At the end, save the summary table
df_global = pd.DataFrame(global_metrics)
df_global.to_excel(os.path.join(root_folder, "all_trials_summary.xlsx"), index=False)
print("Global summary saved to all_trials_summary.xlsx")

Global summary saved to all_trials_summary.xlsx
